In [2]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import create_engine
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
 'pwd': 'Sdf4$2;d-d30pp', # пароль
 'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
 'port': 6432, # порт подключения
 'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
 db_config['pwd'],
 db_config['host'],
 db_config['port'],
 db_config['db'])
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'}) 

# SQL

### Задания

- Посчитайте, сколько книг вышло после 1 января 2000 года;
- Для каждой книги посчитайте количество обзоров и среднюю оценку;
- Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры;
- Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками;
- Посчитайте среднее количество обзоров от пользователей, которые поставили больше 50 оценок.

### Как выполнить задание?

- Опишите цели исследования;
- Исследуйте таблицы — выведите первые строки;
- Сделайте по одному SQL-запросу для решения каждого задания;
- Выведите результат каждого запроса в тетрадке;
- Опишите выводы по каждой из решённых задач.
----------------

Изучим данные

In [6]:
query = ''' SELECT * 
                FROM books
            LIMIT 10'''
pd.io.sql.read_sql(query, con = engine) 

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268
5,6,257,1st to Die (Women's Murder Club #1),424,2005-05-20,116
6,7,258,2nd Chance (Women's Murder Club #2),400,2005-05-20,116
7,8,260,4th of July (Women's Murder Club #4),448,2006-06-01,318
8,9,563,A Beautiful Mind,461,2002-02-04,104
9,10,445,A Bend in the Road,341,2005-04-01,116


In [7]:
query = ''' SELECT * 
                FROM authors
            LIMIT 3'''
pd.io.sql.read_sql(query, con = engine) 

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie


In [8]:
query = ''' SELECT * 
                FROM publishers
            LIMIT 5'''
pd.io.sql.read_sql(query, con = engine) 

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


In [9]:
query = ''' SELECT * 
                FROM ratings
            LIMIT 5'''
pd.io.sql.read_sql(query, con = engine) 

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [10]:
query = ''' SELECT * 
                FROM reviews
            LIMIT 5'''
pd.io.sql.read_sql(query, con = engine) 

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


<div style="border:solid green 2px; padding: 20px"> 
    
Посчитайте, сколько книг вышло после 1 января 2000 года
    
</div>

In [5]:
query = ''' SELECT COUNT(DISTINCT book_id) AS books_count
                FROM books
                WHERE publication_date > '2000-01-01' '''   
pd.io.sql.read_sql(query, con = engine) 

,books_count
0,819


<div style="border:solid green 2px; padding: 20px"> 
    
Для каждой книги посчитайте количество обзоров и среднюю оценку
    
</div>

In [25]:
query = '''
SELECT
    books.title AS title,
    COUNT(DISTINCT reviews.review_id) AS review_count,
    AVG(ratings.rating) AS avg_rating
FROM
    books
    INNER JOIN reviews ON reviews.book_id = books.book_id
    INNER JOIN ratings ON ratings.book_id = books.book_id
GROUP BY
    books.title
ORDER BY
    review_count DESC
LIMIT 7 '''

pd.io.sql.read_sql(query, con = engine) 

,title,review_count,avg_rating
0,Memoirs of a Geisha,8,4.138462
1,Twilight (Twilight #1),7,3.662500
2,The Alchemist,6,3.789474
3,Harry Potter and the Chamber of Secrets (Harry...,6,4.287500
4,Eat Pray Love,6,3.395833
5,Outlander (Outlander #1),6,4.125000
6,Harry Potter and the Prisoner of Azkaban (Harr...,6,4.414634


<div style="border:solid green 2px; padding: 20px"> 
    
Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры
    
</div>

In [18]:
query = '''
SELECT
    publishers.publisher AS publisher,
    COUNT(books.num_pages > 50) AS pages
FROM
    books
    INNER JOIN publishers ON publishers.publisher_id = books.publisher_id

GROUP BY
    publisher
ORDER BY
    pages DESC

LIMIT 3 '''

pd.io.sql.read_sql(query, con = engine)

,publisher,pages
0,Penguin Books,42
1,Vintage,31
2,Grand Central Publishing,25


<div style="border:solid green 2px; padding: 20px"> 
    
Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками
    
</div>

In [19]:
query = ''' 
SELECT
    authors.author as author_name,
    AVG(ratings.rating) as avg_rating
FROM authors
    INNER JOIN books on books.author_id = authors.author_id
    INNER JOIN ratings on ratings.book_id = books.book_id
GROUP BY author_name
    HAVING COUNT(ratings.rating) > 50
ORDER BY avg_rating DESC
LIMIT 5 '''

pd.io.sql.read_sql(query, con = engine)

,author_name,avg_rating
0,J.K. Rowling/Mary GrandPré,4.288462
1,Agatha Christie,4.283019
2,Markus Zusak/Cao Xuân Việt Khương,4.264151
3,J.R.R. Tolkien,4.240964
4,Roald Dahl/Quentin Blake,4.209677


<div style="border:solid green 2px; padding: 20px"> 
    
Посчитайте среднее количество обзоров от пользователей, которые поставили больше 50 оценок
    
</div>

In [23]:
query = '''
SELECT
     SUM(sum.reviewcount)/COUNT(sum.reviewcount) AS avgcountreview
FROM(    
    SELECT reviews.username, 
    COUNT(reviews.username) as reviewcount
    FROM reviews
    WHERE reviews.username in (
                                SELECT ratings.username
                                FROM ratings
                                GROUP BY ratings.username
                                HAVING COUNT(ratings.rating_id) > 50
    
                                )
    GROUP BY username) as sum

'''
pd.io.sql.read_sql(query, con = engine)   

,avgcountreview
0,24.333333


<div style="border:solid lightblue 2px; padding: 20px"> 
    
**Выводы**
    
</div>

- С 1 января 2000 года вышло 819 книг
- Книга с наибольшим количеством ревью – Мемуары гейши, имеет средний рейтинг 4,1
- Издательство, которое выпустило наибольшее число книг толще 50 страниц: Penguin Books
- Автор с самой высокой средней оценкой книг J.K. Rowling, кто бы мог подумать
- Среднее количество обзоров от пользователей, которые поставили больше 50 оценок - 24
